# Amazon QuickSight 구축하기
--------------

### 1. QuickSight 생성하기

AWS 콘솔에서 QuickSight 서비스를 생성합니다. 기본 설정에서 하단의 continue 버튼을 클릭합니다.
<p align="center">
<center><img src="./BlogImages/quicksight_start.png" height="250" width="550" alt=""><center>
<br><br>
</p>  
 
- Region은 편의성을 위해 현재 사용 중인 리전을 선택합니다. 
- 다음 Account Name은 unique한 이름으로 설정합니다. 
- 이메일 주소를 넣습니다.
- Amazon S3를 선택한 다음 default_bucket 버킷명을 찾아서 선택하고 Finish 버튼을 클릭합니다.

In [ ]:
import boto3
import json

In [ ]:
session = boto3.Session()
s3 = session.client(service_name="s3")

In [ ]:
%store -r
%store

In [ ]:
print(bucket)

In [ ]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()['Account']

In [ ]:
iam_client = boto3.client('iam')

In [ ]:
RoleName = f'TeamRole'

iam_client.attach_role_policy(
    RoleName=RoleName,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)
iam_client.attach_role_policy(
    RoleName=RoleName,
    PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
)

    
<p align="center">
<center><img src="./BlogImages/quicksight_setting.png" height="750" width="1050" alt=""><center>
<br><br>
</p>  

 

### 8. QuickSight에서 사용할 manifest_file 생성하기

quicksight에서 사용할 manifest_file 생성합니다.

In [ ]:
manifest_file = {
                "fileLocations": [
                    {
                        "URIPrefixes": [
                            f"s3://{bucket}/amzforecast/results/results_clean.csv"
                        ]
                    }
                ],
                "globalUploadSettings": {
                    "format": "CSV",
                    "delimiter": ",",
                    "textqualifier": "\"",
                    "containsHeader": "true"
                }
            }

In [ ]:
!mkdir ./quicksight

In [ ]:
with open("./quicksight/manifest_file.json", 'w', encoding="utf-8") as f:
    json.dump(manifest_file, f, indent="\t")

   
왼쪽 메뉴에서 **Datasets**를 선택한 다음 오른쪽 상단의 **New dataset**을 선택합니다. 이후 **S3**를 선택하면 아래와 같이 팝업 창이 뜨고, 원하는 이름으로 **data source name**을 설정한 다음, **upload a manifest file**에는 위에 copy한 S3 주소를 입력합니다.

In [ ]:
!aws s3 cp ./quicksight/manifest_file.json s3://$bucket/quicksight/ --quiet
print(f"s3://{bucket}/quicksight/manifest_file.json")

<p align="center">
<center><img src="./BlogImages/quicksight_dataset.png" height="750" width="1050" alt=""><center>
<br><br>
</p>  

앞에서 생성한 dataset에서 **visualize**를 선택하면 아래와 같이 그래프 생성이 가능합니다.

<p align="center">
<center><img src="./BlogImages/quick-vis.png" height="750" width="1050" alt=""><center>
<br><br>
</p>  

QuickSight에 관련된 자세한 실습은 [QuickSight Workshop](https://learnquicksight.workshop.aws/en/author-workshop/0.prerequisites.html) 에서 수행해 보시기 바랍니다.